In [24]:
import boto3
from datetime import datetime
import string

### Display a list of AWS S3 buckets

In [2]:
# Create an S3 client
s3 = boto3.client('s3')

In [3]:
# Call S3 to list current buckets
response = s3.list_buckets()

In [4]:
# Get a list of all bucket names from the response
buckets = [bucket['Name'] for bucket in response['Buckets']]

In [5]:
# Print out the bucket list
print("Bucket List: %s" % buckets)

Bucket List: ['road-trips']


### Run text detection/OCR on AWS Rekognition

In [6]:
bucket = 'road-trips'
photo = 'frame46.jpg'

client = boto3.client('rekognition')
response = client.detect_text(Image={'S3Object':{'Bucket':bucket,'Name':photo}})
textDetections = response['TextDetections']
print(response)
print('Detected Text')
for text in textDetections:
    print('Detected text:' + text['DetectedText'])
    print('Confidence:' + "{:.2f}".format(text['Confidence']) + '%')
    print('Id: {}'.format(text['Id']))
    if 'ParentId' in text:
        print('Parent Id: {}'.format(text['ParentId']))
    print('Type:' + text['Type'])
    print()

{'TextDetections': [{'DetectedText': 'DISTRICT OF COLUMBIA', 'Type': 'LINE', 'Id': 0, 'Confidence': 99.83846282958984, 'Geometry': {'BoundingBox': {'Width': 0.8551766872406006, 'Height': 0.08189552277326584, 'Left': 0.12758195400238037, 'Top': 0.06818182021379471}, 'Polygon': [{'X': 0.12758195400238037, 'Y': 0.06818182021379471}, {'X': 0.982758641242981, 'Y': 0.06810447573661804}, {'X': 0.98276287317276, 'Y': 0.15000000596046448}, {'X': 0.1275862157344818, 'Y': 0.15007734298706055}]}}, {'DetectedText': 'ELEVATION: 26 M', 'Type': 'LINE', 'Id': 1, 'Confidence': 99.70556640625, 'Geometry': {'BoundingBox': {'Width': 0.5932831764221191, 'Height': 0.0790286436676979, 'Left': 0.12759043276309967, 'Top': 0.37097057700157166}, 'Polygon': [{'X': 0.12759043276309967, 'Y': 0.37097057700157166}, {'X': 0.7208735942840576, 'Y': 0.3734480142593384}, {'X': 0.7206836938858032, 'Y': 0.4524766504764557}, {'X': 0.1274005025625229, 'Y': 0.44999921321868896}]}}, {'DetectedText': 'HEADING: 127 (T)', 'Type': '

In [31]:
bucket = 'road-trips'
photo = 'frame46.jpg'

client = boto3.client('rekognition')
response = client.detect_text(Image={'S3Object':{'Bucket':bucket,'Name':photo}})
textDetections = response['TextDetections']

loc_time_dict = {}

for text in textDetections:
    if text['Type'] == 'LINE':
        if text['Id'] == 0:
            loc_time_dict['State'] = text['DetectedText']
        elif text['Id'] == 1:
            if text['DetectedText'].startswith('ELEVATION'):
                loc_time_dict['Elev'] = int(text['DetectedText'].split(' ')[1])
            else:
                loc_time_dict['County'] = text['DetectedText'].split(' ')[0]
        elif text['Id'] == 2:
            if text['DetectedText'].startswith('ELEVATION'):
                loc_time_dict['Elev'] = int(text['DetectedText'].split(' ')[1])
            else:
                loc_time_dict['Heading'] = int(text['DetectedText'].split(' ')[1])
        elif text['Id'] == 3:
            if text['DetectedText'].startswith('HEADING'):
                loc_time_dict['Heading'] = int(text['DetectedText'].split(' ')[1])
            else:
                loc_time_dict['DateTime'] = datetime.strptime(text['DetectedText'].split(' ')[1],'%m/%d/%y')
        elif text['Id'] == 4:
            if text['DetectedText'].startswith('DATE'):
                loc_time_dict['DateTime'] = datetime.strptime(text['DetectedText'].split(' ')[1],'%m/%d/%y')
            else:
                loc_time_dict['DateTime'] = datetime.combine(loc_time_dict['DateTime'],
                                                             datetime.strptime(text['DetectedText'][6:],'%I:%M:%S %p').time())
        elif text['Id'] == 5:
            loc_time_dict['DateTime'] = datetime.combine(loc_time_dict['DateTime'],
                                                         datetime.strptime(text['DetectedText'][6:],'%I:%M:%S %p').time())
            
print(loc_time_dict)

{'State': 'DISTRICT OF COLUMBIA', 'Elev': 26, 'Heading': 127, 'DateTime': datetime.datetime(2018, 5, 3, 5, 54, 27)}


In [23]:
image_dict = {}

bucket = 'road-trips'
photo = 'frame21.jpg'

client = boto3.client('rekognition')
response = client.detect_text(Image={'S3Object':{'Bucket':bucket,'Name':photo}})
textDetections = response['TextDetections']

GPS_dict = {}
for text in textDetections:
    if text['Type'] == 'LINE':
        remove = string.punctuation + string.ascii_uppercase
        remove = remove.replace(".", "") # don't remove periods
        if text['Id'] == 0:
            GPS_dict['Lat'] = text['DetectedText'].translate({ord(char): None for char in remove})
            GPS_dict['Lat'] = (float(GPS_dict['lat'].split(' ')[0]) + float(GPS_dict['lat'].split(' ')[1])/60 +
            float(GPS_dict['Lat'].split(' ')[2])/3600)
        elif text['Id'] == 1:
            GPS_dict['Lon'] = text['DetectedText'].translate({ord(char): None for char in remove})
            GPS_dict['Lon'] = (float(GPS_dict['lon'].split(' ')[0]) + float(GPS_dict['lon'].split(' ')[1])/60 +
            float(GPS_dict['Lon'].split(' ')[2])/3600)

print(GPS_dict)

#text = text.translate(str.maketrans('','','1234567890'))
# loc_time_dict = {}
# for t
            
# image_dict[photo.split('.')[0]] 

{'lat': 38.90211111111111, 'lon': 77.04916666666666}


In [7]:
bucket = 'road-trips'
photo = 'frame21.jpg'

client = boto3.client('rekognition')
response = client.detect_text(Image={'S3Object':{'Bucket':bucket,'Name':photo}})
textDetections = response['TextDetections']
print(response)
print('Detected Text')
for text in textDetections:
    print('Detected text:' + text['DetectedText'])
    print('Confidence:' + "{:.2f}".format(text['Confidence']) + '%')
    print('Id: {}'.format(text['Id']))
    if 'ParentId' in text:
        print('Parent Id: {}'.format(text['ParentId']))
    print('Type:' + text['Type'])
    print()

{'TextDetections': [{'DetectedText': 'N38 54\' 07.6"', 'Type': 'LINE', 'Id': 0, 'Confidence': 98.52267456054688, 'Geometry': {'BoundingBox': {'Width': 0.8652546405792236, 'Height': 0.30393847823143005, 'Left': 0.06487680971622467, 'Top': 0.11052931100130081}, 'Polygon': [{'X': 0.06487680971622467, 'Y': 0.11052931100130081}, {'X': 0.9301314353942871, 'Y': 0.11940710991621017}, {'X': 0.9297224283218384, 'Y': 0.42334556579589844}, {'X': 0.06446778029203415, 'Y': 0.41446778178215027}]}}, {'DetectedText': 'W77 02\' 57.0"', 'Type': 'LINE', 'Id': 1, 'Confidence': 99.6412582397461, 'Geometry': {'BoundingBox': {'Width': 0.9032117128372192, 'Height': 0.3182213604450226, 'Left': 0.0540621392428875, 'Top': 0.5325180888175964}, 'Polygon': [{'X': 0.0540621392428875, 'Y': 0.5325180888175964}, {'X': 0.9572738409042358, 'Y': 0.5441309809684753}, {'X': 0.9567371606826782, 'Y': 0.8623523116111755}, {'X': 0.05352549999952316, 'Y': 0.8507394790649414}]}}, {'DetectedText': 'N38', 'Type': 'WORD', 'Id': 2, 'P